In [1]:
import pandas as pd
import numpy as np
from BK_Tree import BKTree
import re
import string
from dead_processing_utils import *
from file_processing_utils import *

                                                 string  \
97                                               Arabia   
251                               Brazil, South America   
34                                               Alaska   
1569                                              Natal   
175                                                Bali   
...                                                 ...   
1998  Santa Catalina Island, Los Angeles County, Cal...   
1997  Santa Barbara Island, Santa Barbara County, Ca...   
195   Bazhong Prefecture, Nanjiang County, Sichuan P...   
1517  Municipio San Cristobal de las Casas, Chiapas ...   
1515  Municipio Pueblo Nuevo Solistahuacan, Chiapas ...   

                                        stripped_string  score  
97                                               arabia    3.0  
251                                             brazil     3.0  
34                                               alaska    4.0  
1569                           

### Load in all the dead records as well as the determination key from the db

In [3]:
folder_path = 'dead_records_data'
input_df = concat_df(folder_path)
print(input_df.columns)

determination_key = pd.read_csv("data/NoAuthorTest.csv")
determination_key['no_author'] = determination_key['no_author'].apply(lambda x: punctStrip(x))

Index(['media handling record: fileName', 'taxon', 'Taxon record: family',
       'objectNumber', 'Taxon record: naturalHistoryCommonName',
       'fieldLocCountry', 'briefDescription', 'fieldCollector',
       'fieldCollectionNumber', 'fieldCollectionDateGroup', 'source',
       'accessionDate', 'Voucher record: Wild',
       'Voucher record: Horticultural', 'reference',
       'referenceGroup_reference', 'Distribution record: depositor_Plant',
       'Distribution record: depositor_Seed', 'notes', 'deadFlag',
       'flowerColor', 'comment'],
      dtype='object')


### Determine the amount of dead card taxon already in the database verbatim 

In [4]:
input_df = TaxonNoAuthor(input_df, 'taxon')
input_df['***no_author'] = input_df['***no_author'].apply(lambda x: punctStrip(x))
input_df['is_cultivar'] = input_df['***no_author'].apply(lambda x: detectCultivar(x))
input_df['***no_author'] = input_df.apply(lambda x: x['***no_author'].split()[0] if x['is_cultivar'] else x['***no_author'], axis=1)

In [ ]:
input_df['taxon_missing'] = input_df['***no_author'].apply(lambda x: True if x not in determination_key['no_author'].values else False)
taxon_to_add = input_df[input_df['taxon_missing']]['***no_author'].unique()
taxon_to_add = input_df[input_df['***no_author'].isin(taxon_to_add)]
already_in_db = input_df[~input_df['taxon_missing']]

print("Total taxon: {}".format(len(input_df['***no_author'].unique())))
print("Taxon not in database verbatim: {}".format(len(taxon_to_add)))

Total taxon: 2
Taxon not in database verbatim: 9184


### Create a best guess taxon key using the Bk Tree

In [ ]:
taxaTree = BKTree()
for name in determination_key['no_author']:
    taxaTree.insert(name)

In [ ]:
taxon_to_add['recommendation'] = taxon_to_add['***no_author'].apply(lambda x: sortOutput(taxaTree.search(x, 2)))

In [ ]:
taxon_to_add.to_csv("output/taxon_to_add.csv", index=False)

### Create a complete key with both taxn guesses and known taxon already in the db